In [4]:
import os, sys
sys.path.append("../")
import omegaconf
from torch_points3d.datasets.segmentation.s3dis import S3DISOriginalFused

In [5]:
tp3d_dataset_path = "/home/obarads/databox/torch_point3d/"

In [2]:
from torch_geometric.data import InMemoryDataset

def get_paths(root, areas, extension=".h5"):
    assert extension in [".h5", ".npy"]

    area_list = []
    for i in areas:
        area_list.append(f"Area_{i}")

    h5_file_path_list = []

    for area in area_list:
        paths = glob.glob(os.path.join(root, area+"*"+extension))
        for h5_file_path in sorted(paths):
            if h5_file_path.endswith(extension):
                h5_file_path_list.append(h5_file_path)

    return h5_file_path_list

class S3DIS1x1Ins(InMemoryDataset):
    r"""The (pre-processed) Stanford Large-Scale 3D Indoor Spaces dataset from
    the `"3D Semantic Parsing of Large-Scale Indoor Spaces"
    <http://buildingparser.stanford.edu/images/3D_Semantic_Parsing.pdf>`_
    paper, containing point clouds of six large-scale indoor parts in three
    buildings with 12 semantic elements (and one clutter class).

    Args:
        root (string): Root directory where the dataset should be saved.
        test_area (int, optional): Which area to use for testing (1-6).
            (default: :obj:`6`)
        train (bool, optional): If :obj:`True`, loads the training dataset,
            otherwise the test dataset. (default: :obj:`True`)
        transform (callable, optional): A function/transform that takes in an
            :obj:`torch_geometric.data.Data` object and returns a transformed
            version. The data object will be transformed before every access.
            (default: :obj:`None`)
        pre_transform (callable, optional): A function/transform that takes in
            an :obj:`torch_geometric.data.Data` object and returns a
            transformed version. The data object will be transformed before
            being saved to disk. (default: :obj:`None`)
        pre_filter (callable, optional): A function that takes in an
            :obj:`torch_geometric.data.Data` object and returns a boolean
            value, indicating whether the data object should be included in the
            final dataset. (default: :obj:`None`)
    """

    url = ('https://shapenet.cs.stanford.edu/media/'
           'indoor3d_sem_seg_hdf5_data.zip')

    def __init__(self,
                 root,
                 test_area=6,
                 train=True,
                 transform=None,
                 pre_transform=None,
                 pre_filter=None):
        assert test_area >= 1 and test_area <= 6
        self.test_area = test_area
        super(S3DIS1x1Ins, self).__init__(root, transform, pre_transform, pre_filter)
        path = self.processed_paths[0] if train else self.processed_paths[1]
        self.data, self.slices = torch.load(path)

    @property
    def raw_file_names(self):
        return ['room_filelist.txt']
        

    @property
    def processed_file_names(self):
        test_area = self.test_area
        return ['{}_{}.pt'.format(s, test_area) for s in ['train', 'test']]

    def download(self):
        pass

    def process(self):
        # train data
        ## get train data path
        train_area = [i for i in range(1,7) if i != self.test_area]
        train_file_paths = get_paths(osp.join(self.raw_dir, "blocks"), train_area)
        ## get train data
        train_xs, train_sem_labels, train_ins_labels = [], [], []
        for file_path in train_file_paths:
            f = h5py.File(file_path)
            train_xs += torch.from_numpy(f['data'][:]).unbind(0)
            train_sem_labels += torch.from_numpy(f['seglabel'][:]).to(torch.long).unbind(0)
            train_ins_labels += torch.from_numpy(f["pid"][:]).to(torch.long).unbind(0)
        # pre_transform and filter
        train_data_list = []
        for i in range(len(train_xs)):
            data = Data(pos=train_xs[i][:, :3], x=train_xs[i][:, 3:])
            data.y = train_sem_labels[i]
            data.ins = train_ins_labels[i]
            if self.pre_filter is not None and not self.pre_filter(data):
                continue
            if self.pre_transform is not None:
                data = self.pre_transform(data)
            train_data_list.append(data)

        torch.save(self.collate(train_data_list), self.processed_paths[0])

        # test data
        ## get test data path
        test_file_paths = get_paths(osp.join(self.raw_dir, "blocks"), [self.test_area])
        test_xs, test_sem_labels, test_ins_labels = [], [], []
        ## get test data
        for file_path in test_file_paths:
            f = h5py.File(file_path)
            test_xs += torch.from_numpy(f['data'][:]).unbind(0)
            test_sem_labels += torch.from_numpy(f['seglabel'][:]).to(torch.long).unbind(0)
            test_ins_labels += torch.from_numpy(f["pid"][:]).to(torch.long).unbind(0)
        # pre_transform and filter
        test_data_list = []
        for i in range(len(test_xs)):
            data = Data(pos=test_xs[i][:, :3], x=test_xs[i][:, 3:])
            data.y = test_sem_labels[i]
            data.ins = test_ins_labels[i]
            if self.pre_filter is not None and not self.pre_filter(data):
                continue
            if self.pre_transform is not None:
                data = self.pre_transform(data)
            test_data_list.append(data)

        torch.save(self.collate(test_data_list), self.processed_paths[1])


In [4]:
from torch_points3d.datasets.base_dataset import BaseDataset

class S3DIS1x1InsDataset(BaseDataset):
    def __init__(self, dataset_opt):
        super().__init__(dataset_opt)

        pre_transform = self.pre_transform
        self.train_dataset = S3DIS1x1Ins(
            self._data_path,
            test_area=self.dataset_opt.fold,
            train=True,
            pre_transform=self.pre_transform,
            transform=self.train_transform,
        )
        self.test_dataset = S3DIS1x1Ins(
            self._data_path,
            test_area=self.dataset_opt.fold,
            train=False,
            pre_transform=pre_transform,
            transform=self.test_transform,
        )
        if dataset_opt.class_weight_method:
            self.add_weights(class_weight_method=dataset_opt.class_weight_method)

    def get_tracker(self, wandb_log: bool, tensorboard_log: bool):
        """Factory method for the tracker

        Arguments:
            wandb_log - Log using weight and biases
            tensorboard_log - Log using tensorboard
        Returns:
            [BaseTracker] -- tracker
        """
        from torch_points3d.metrics.segmentation_tracker import SegmentationTracker

        return SegmentationTracker(self, wandb_log=wandb_log, use_tensorboard=tensorboard_log)


ModuleNotFoundError: No module named 'torch_points3d'